# Формирование таблицы словаря 'Users' для базы данных.

### 1. Подготовка

In [ ]:

import os
import time
import json
import datetime

import requests as rq
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine


### 2. Формирование переменных для подключения

In [ ]:

KEY = ''
TOKEN = 'Bearer ' + KEY
API = '/api/users/getall'
URL = 'https://tenderplan.ru' + API
headers = {
    'Authorization': TOKEN,
    'Accept': 'application/json'
}

engine_pet = create_engine('mysql+mysqlconnector://root:''@''/pet_proect')


### 3. Подключение к API tenderplan

In [64]:

response_users = rq.get(
    URL,
    headers = headers
).json()


### 4. Формируем функцию для преобразования UNIX-времени в правильный формат datetime

In [56]:

if __name__ == '__main__':
    def f_date_fr_time(date_):
        '''Принимает UNIX-время и возвращате дату в формате datetime'''
        return datetime.datetime.fromtimestamp(date_//1000)
        

### 5. Формирование DataFrame 'Users' и преобразовываем в нужный вид

Формируем DataFrame с columns:
- _id - уникальный ключ сотрудников компании;
- email - почта сотрудников компании;
- phoneNumber - телефон сотрудников компании;
- displayName - ФИО сотрудников компании;
- firmId - id компании;
- createDateTime - дата заведения сотрудника (формат UNXI-время);
- isDeleted - удален ли сотрудник из базы.


In [66]:

df_users = (
    pd.DataFrame(
        response_users,
        columns = [
            '_id',
            'email',
            'phoneNumber',
            'displayName',
            'firmId',
            'createDateTime',
            'isDeleted'
        ]
    )
    .set_index('_id')
)

df_users = df_users.assign(
    **{
        'createDateTime': df_users['createDateTime'].map(f_date_fr_time),
        'phoneNumber': df_users['phoneNumber'].fillna(np.nan).astype(np.float32)
    }
)



### 6. Загружаем данные во временную таблицу 'tmp_dict_users'

In [40]:

(
    df_users
    .reset_index()
    .to_sql(
        'tmp_dict_users', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)


-1

In [65]:
df_users.head()

email   phoneNumber  \
_id                                                                     
63c8d71004a28500ff75d0a8        office@energotrade42.ru  8.906985e+10   
63c8e6c304a28500ff75e609  s.chernousov@energotrade42.ru  8.995126e+10   
64631f682a48ecc1d1e4f25d           img@energotrade42.ru           NaN   
6474472b726fb3576edb9811           ikt@energotrade42.ru           NaN   
64906d2e7a3fac2f79cd95e0            shin-prof@yandex.ru           NaN   

                                  displayName                    firmId  \
_id                                                                       
63c8d71004a28500ff75d0a8       Осадчий Никита  63c8d71004a28500ff75d0a7   
63c8e6c304a28500ff75e609  Станислав Черноусов  63c8d71004a28500ff75d0a7   
64631f682a48ecc1d1e4f25d          Марк Иванов  63c8d71004a28500ff75d0a7   
6474472b726fb3576edb9811      Кира Ибрагимова  63c8d71004a28500ff75d0a7   
64906d2e7a3fac2f79cd95e0              Шинпроф  63c8d71004a28500ff75d0a7   

                              createDateTime  isDeleted  
_id                                                      
63c8d71004a28500ff75d0a8 2023-01-19 12:37:20      False  
63c8e6c304a28500ff75e609 2023-01-19 13:44:19      False  
64631f682a48ecc1d1e4f25d 2023-05-16 13:15:04      False  
6474472b726fb3576edb9811 2023-05-29 13:33:15      False  
64906d2e7a3fac2f79cd95e0 2023-06-19 21:58:54       True